In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [2]:
rating = pd.read_csv('/Users/blakemyers/Desktop/data/ratings.csv', error_bad_lines=False, encoding='latin-1')

In [3]:
movie = pd.read_csv("/Users/blakemyers/Desktop/data/movies.csv", error_bad_lines=False, encoding='latin-1')

In [4]:
movie_rating = pd.merge(rating, movie, on = 'movieId')

In [5]:
cols = ['timestamp']

In [6]:
movie_rating.drop(cols, axis=1, inplace=True)

In [7]:
numrate_movie = movie_rating.groupby("title")["rating"].count().reset_index()

In [8]:
numrate_movie.rename({"rating": "ratecount_movie"}, axis=1, inplace=True)

In [9]:
numrate_movie = numrate_movie.query("ratecount_movie >= 20")

In [10]:
ratings20plus = pd.merge(numrate_movie, movie_rating, on = 'title', how = 'inner')

In [11]:
numrate_user = ratings20plus.groupby("userId")["rating"].count().reset_index()

In [12]:
numrate_user.rename({"rating": "ratecount_user"}, axis=1, inplace=True)

In [13]:
numrate_user = numrate_user.query("ratecount_user >= 20")

In [14]:
ur20plus = pd.merge(ratings20plus, numrate_user, on = "userId", how = "inner")

In [15]:
ur20plus.head()

,title,ratecount_movie,userId,movieId,rating,genres,ratecount_user
0,(500) Days of Summer (2009),42,15,69757,4.0,Comedy|Drama|Romance,122
1,101 Dalmatians (One Hundred and One Dalmatians...,44,15,2085,1.5,Adventure|Animation|Children,122
2,28 Days Later (2002),58,15,6502,3.5,Action|Horror|Sci-Fi,122
3,A.I. Artificial Intelligence (2001),56,15,4370,4.0,Adventure|Drama|Sci-Fi,122
4,"Adjustment Bureau, The (2011)",21,15,84954,4.5,Romance|Sci-Fi|Thriller,122


In [16]:
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled

In [17]:
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)

In [18]:
user_movie_matrix.head()

title,(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),2001: A Space Odyssey (1968),2012 (2009),...,Young Frankenstein (1974),Young Guns (1988),Zack and Miri Make a Porno (2008),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),Â¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.777778,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.777778
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.555556,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [19]:
X_train, X_test = train_test_split(user_movie_matrix, train_size=0.8)

In [20]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
num_input = ur20plus['title'].nunique()
n_nodes_inpl = num_input  
n_nodes_hl1  = 256  
n_nodes_outl = num_input  
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl]))}

In [22]:
input_layer = tf.placeholder('float', [None, num_input])
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,hidden_1_layer_vals['weights']))
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
output_true = tf.placeholder('float', [None, num_input])
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
learn_rate = 0.1
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 100
hm_epochs =200
tot_images = X_train.shape[0]

In [24]:
for epoch in range(hm_epochs):
    epoch_loss = 0
    
    for i in range(int(tot_images/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, \
               output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer,\
               feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer,\
                   feed_dict={input_layer:X_test})
        
    if epoch <= 5 or epoch >= 195:
        print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))
        print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)
    elif epoch == 11:
        print('.......................')
    else:
        continue

MSE train 83.75465981526779 MSE test 84.85296906274816
Epoch 0 / 200 loss: 385.3652648925781
MSE train 70.68105957100396 MSE test 72.14828904171337
Epoch 1 / 200 loss: 314.57279205322266
MSE train 61.91727505136641 MSE test 63.62368378249938
Epoch 2 / 200 loss: 269.13724517822266
MSE train 55.72632158351596 MSE test 57.55697368904683
Epoch 3 / 200 loss: 238.14786911010742
MSE train 51.12612202749976 MSE test 53.013614837195014
Epoch 4 / 200 loss: 215.84630966186523
MSE train 47.58427961698249 MSE test 49.51581215997028
Epoch 5 / 200 loss: 199.04828262329102
.......................
MSE train 4.836489189678325 MSE test 7.8124144061933345
Epoch 195 / 200 loss: 18.040662050247192
MSE train 4.804673743405435 MSE test 7.776397753747008
Epoch 196 / 200 loss: 17.913559198379517
MSE train 4.773234785361022 MSE test 7.740823663008395
Epoch 197 / 200 loss: 17.78796935081482
MSE train 4.742167508883588 MSE test 7.705752549498793
Epoch 198 / 200 loss: 17.663816213607788
MSE train 4.711475880663064 

In [25]:
userCol = ur20plus["userId"]
max_value = userCol.max()
new_userId = max_value + 1
ur20plus = ur20plus.append(pd.DataFrame([["Hulk (2003)",1,new_userId,1,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)

In [26]:
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)

In [27]:
top_ten_ranked.loc[top_ten_ranked['userId'] == new_userId] #an example of the top 10 recommendations for the above new user who only rated one movie, i.e., "Hulk (2003)", with a 5/5.

,userId,title,rating
734111,611,2012 (2009),9.903190
734497,611,Fantasia (1940),9.821529
734145,611,Alice in Wonderland (1951),9.188251
734305,611,Burn After Reading (2008),7.678481
734715,611,James and the Giant Peach (1996),7.655343
734326,611,Charlie and the Chocolate Factory (2005),7.221748
734845,611,Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (...,6.947073
734792,611,"Lives of Others, The (Das leben der Anderen) (...",6.881359
734520,611,First Knight (1995),6.547202
734989,611,Pleasantville (1998),6.454093


In [28]:
userCol = ur20plus["userId"]
max_value = userCol.max()
new_userId = max_value + 1
ur20plus = ur20plus.append(pd.DataFrame([["(500) Days of Summer (2009)",1,new_userId,1,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)
ur20plus = ur20plus.append(pd.DataFrame([["Aliens (1986)",1,new_userId,1,3,"genre",1]], columns =ur20plus.columns), ignore_index=True)
ur20plus = ur20plus.append(pd.DataFrame([["Brave (2012)",1,new_userId,1,4,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)

In [29]:
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)

In [30]:
top_ten_ranked.loc[top_ten_ranked['userId'] == new_userId] #an example of the top 10 recommendations for the above new user who rated three movies, i.e., "(500) Days of Summer (2009)" with a 5/5, "Aliens (1986)" with a 3/5, and "Brave (2012)" with a 4/5.

,userId,title,rating
735623,612,Charlie and the Chocolate Factory (2005),9.378066
735624,612,Charlie's Angels (2000),8.660883
735518,612,Beauty and the Beast (1991),7.908426
736409,612,Shakespeare in Love (1998),7.771241
736616,612,Up (2009),7.756080
736634,612,Wallace & Gromit: A Close Shave (1995),7.283265
736097,612,"Lord of the Rings: The Return of the King, The...",7.257791
735729,612,Do the Right Thing (1989),7.246536
735912,612,Halloween (1978),7.135806
736093,612,"Long Kiss Goodnight, The (1996)",6.850907
